In [ ]:
!pip install -q yt-dlp==2025.3.31
!pip install -q faster-whisper==1.1.1
!pip install -q ffmpeg-python==0.2.0
!pip install -q chromadb==1.0.6
!pip install -q langchain==0.3.24
!pip install -q langchain-community==0.3.22
!pip install -q transformers==4.51.3
!pip install -q bitsandbytes==0.45.5
!pip install -q huggingface_hub==0.30.2
!pip install -q triton==3.1.0
!pip install -q sentence-transformers==4.1.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.2/172.2 kB 4.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 48.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.8 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.2/35.2 MB 56.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 47.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 80.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 69.4 MB/s eta 0:00:00:00:0100:01
   ━━━

In [ ]:
!huggingface-cli login --token "TOKEN"
!huggingface-cli whoami

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
The token `llm_models` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `llm_models`
dungntn


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
import torch

print(torch.__version__)
print(torch.cuda.is_available())

2.5.1+cu124
True


In [ ]:
import torch

device = f"cuda" if torch.cuda.is_available() else "cpu"
print(device)
dtype = "int8" if device == "cpu" else "float16"
print(dtype)

cuda
float16


In [ ]:
import shutil
import os


def cleanup(folder):
    if os.path.exists(folder):
        shutil.rmtree(folder, ignore_errors=False)
        print(f"Deleted {folder}.")
    else:
        print(f"{folder} doesn't exist.")

# Extract audio from youtube video

In [ ]:
import yt_dlp
import os
import ffmpeg
import uuid


# !yt-dlp --extract-audio --audio-format mp3 -o video_audio.mp3 https://www.youtube.com/watch?v=9RhWXPcKBI8
def extract_audio(video_url) -> str:
    try:
        downloads = "downloads"
        os.makedirs(downloads, exist_ok=True)

        uid = str(uuid.uuid4())
        audio_path = os.path.join(downloads, f"{uid}.%(ext)s")

        if video_url.startswith(("http://", "https://", "www.")):
            print("Downloading audio from YouTube...")
            ydl_opts = {
                "format": "bestaudio/best",
                "outtmpl": audio_path,
                "quiet": True,
                "no_warnings": True,
                "postprocessors": [
                    {
                        "key": "FFmpegExtractAudio",
                        "preferredcodec": "mp3",
                        "preferredquality": "192",
                    }
                ],
                "noplaylist": True,
                "extractaudio": True,
                "writeinfojson": False,
                "writesubtitles": False,
            }

            with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                error_code = ydl.download([video_url])
            if error_code:
                raise Exception("Failed to download audio from YouTube.")

        else:
            print("Extracting audio from local file...")
            if not os.path.isfile(video_url):
                raise FileNotFoundError("Not found video.")
            try:
                (
                    ffmpeg.input(video_url)
                    .output(
                        audio_path, vn=None, acodec="libmp3lame", audio_bitrate="192k"
                    )
                    .run()
                )
            except Exception as e:
                raise Exception(f"Failed to extract audio from local file: {str(e)}")

        # Audio Segment
        audio_path = os.path.join(downloads, f"{uid}.mp3")
        output_dir = "audio"
        os.makedirs(output_dir, exist_ok=True)
        try:
            (
                ffmpeg.input(audio_path)
                .output(
                    os.path.join(output_dir, "chunk_%03d.mp3"),
                    f="segment",
                    segment_time=120,
                    c="copy",
                )
                .run(overwrite_output=True)
            )
        except Exception as e:
            raise Exception(f"Failed to segment audio: {str(e)}")

        # cleanup(downloads)
        return output_dir

    except FileNotFoundError as e:
        print(e)
        return None
    except Exception as e:
        print(e)
        return None


# --- Example ---
# video_url = "/content/drive/MyDrive/DungNTN/English Presentation #shorts #english #englishmerl #englishreading #englishgrammar #education.mp4"
# video_url = "https://www.youtube.com/watch?v=AkDNWw5wQwA"
video_url = "https://www.youtube.com/watch?v=9RhWXPcKBI8"

output_dir = extract_audio(video_url)
print("Audio saved at:", output_dir)

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

Audio saved at: audio


[segment @ 0x5ccadc7c1440] Opening 'audio/chunk_013.mp3' for writing
size=N/A time=00:27:07.48 bitrate=N/A speed=2.57e+03x    
video:0kB audio:38145kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: unknown


# Extract transcript

In [ ]:
max_workers = min(32, os.cpu_count() * 5)
print(f"Max workers: {max_workers}")

Max workers: 20


In [ ]:
import os
import time
from datetime import timedelta
from faster_whisper import WhisperModel
from concurrent.futures import ThreadPoolExecutor

model = WhisperModel("base", device=device, compute_type=dtype)
SEGMENT_TIME = 120


def format_timestamp(seconds):
    return str(timedelta(seconds=int(seconds))).rjust(8, "0")


def transcribe(filename):
    print(f"Transcribing: {filename}\n")
    chunk_index = int(filename.split("_")[1].split(".")[0])
    offset = chunk_index * SEGMENT_TIME

    segments, _ = model.transcribe(os.path.join(output_dir, filename))
    # return ''.join(segment.text for segment in segments)
    transcript = ""
    for segment in segments:
        start = format_timestamp(segment.start + offset)
        end = format_timestamp(segment.end + offset)
        transcript += f"[{start} - {end}] {segment.text.strip()}\n"

    return transcript


def extract_transcript(output_dir):
    files = sorted(
        os.listdir(output_dir), key=lambda x: int(x.split("_")[1].split(".")[0])
    )

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        results = list(executor.map(transcribe, files))

    transcript = "\n".join(results)

    # cleanup(output_dir)
    return transcript


# --- Example ---
t1 = time.time()
transcript = extract_transcript(output_dir)
print(f"Total time: {time.time() - t1:.2f}s")

Transcribing: chunk_000.mp3

Transcribing: chunk_001.mp3

Transcribing: chunk_002.mp3

Transcribing: chunk_003.mp3

Transcribing: chunk_004.mp3

Transcribing: chunk_005.mp3

Transcribing: chunk_006.mp3

Transcribing: chunk_007.mp3

Transcribing: chunk_008.mp3

Transcribing: chunk_009.mp3

Transcribing: chunk_010.mp3

Transcribing: chunk_011.mp3

Transcribing: chunk_012.mp3

Transcribing: chunk_013.mp3

Total time: 46.61s


# Chunking

In [ ]:
from langchain.schema import Document

document = Document(page_content=transcript, metadata={"source": "transcript"})

In [ ]:
from langchain.text_splitter import (
    CharacterTextSplitter,
    RecursiveCharacterTextSplitter,
)


def chunking(documents):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000, chunk_overlap=200, separators=["\n\n", "\n", ".", " "]
    )

    chunks = text_splitter.split_documents(documents)
    return chunks


# --- Example ---
chunks = chunking([document])
print(len(chunks))
for i, chunk in enumerate(chunks):
    print(f"--- Chunk {i} ---\n{chunk.page_content}\n")

53
--- Chunk 0 ---
[00:00:00 - 00:00:09] I built a massive isolation chamber and we're gonna see if these two strangers can survive in this chamber for the next 100 days
[00:00:09 - 00:00:19] They have never met each other ever Bailey. This is Susie. Susie. This is Bailey. Nice to meet you. If the two of you can survive the next 100 days in here
[00:00:19 - 00:00:22] I will give you the half a million dollars inside of this bowl
[00:00:22 - 00:00:26] But if one of you leaves before the hundred days is up you both get nothing
[00:00:27 - 00:00:31] All right, I think you guys understand the rules have fun. Okay, bye
[00:00:32 - 00:00:34] This is gonna be crazy
[00:00:39 - 00:00:45] Yeah, this is actually like an insane asylum. They're currently looking at all the stuff we put in there
[00:00:45 - 00:00:47] We gave them enough food for a hundred days, which is healthy
[00:00:47 - 00:00:50] But basically the exact same thing over and over again

--- Chunk 1 ---
[00:00:45 - 00:00:47] We gav

# Vector database

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

# sentence-transformers/all-MiniLM-L6-v2
# sentence-transformers/all-mpnet-base-v2

embedding_model = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-small")

/tmp/ipykernel_31/3395895694.py:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(
2025-04-24 09:37:14.103484: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745487434.245564      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745487434.284808      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has 

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/498k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [ ]:
print(f"Model device: {embedding_model.client.device}")

Model device: cuda:0


In [ ]:
import os
from langchain.vectorstores import FAISS, Chroma

# vectorstore = FAISS.from_documents(chunks, embedding_model)

persist_directory = "./chroma_db8"

vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embedding_model,
    persist_directory=persist_directory,
)

print("Number of documents in vectorstore:", vectorstore._collection.count())

Number of documents in vectorstore: 53


In [ ]:
retriever = vectorstore.as_retriever()

# --- Example ---
query = "Who are the Carl and Chris?"
docs = retriever.get_relevant_documents(query)

for doc in docs:
    doc_details = doc.to_json()["kwargs"]
    print("Source: ", doc_details["metadata"]["source"])
    print("Text: ", doc_details["page_content"], "\n")

Source:  transcript
Text:  [00:00:45 - 00:00:47] We gave them enough food for a hundred days, which is healthy
[00:00:47 - 00:00:50] But basically the exact same thing over and over again
[00:00:50 - 00:00:56] We also gave them their own private bathroom which comes with the shower and obviously has no cameras inside and a bed to sleep on
[00:00:56 - 00:01:01] They have everything they need to survive a hundred days. It's just a question of do they want it? I got a comb
[00:01:01 - 00:01:03] I don't know if I've ever combed my hair my life
[00:01:05 - 00:01:11] Interesting see how they're both standing on different sides of the room. They're so awkward when I had them take their blindfolds off
[00:01:11 - 00:01:13] That was legitimately the first time they had ever met
[00:01:15 - 00:01:21] I just keep thinking of the feeling like of eggs in it. Oh, yeah, how good is that gonna feel?
[00:01:21 - 00:01:23] Oh 

Source:  transcript
Text:  [00:15:20 - 00:15:22] Is either going to be the b

In [ ]:
# # --- Example ---
# query = "Who are the participants?"
# docs = vectorstore.similarity_search(query)

# print(f"Query: {query}")
# print(f"Retrieved documents: {len(docs)}")
# for doc in docs:
#     doc_details = doc.to_json()['kwargs']
#     print("Source: ", doc_details['metadata']['source'])
#     print("Text: ", doc_details['page_content'], "\n")

# LLM model

In [ ]:
from transformers import BitsAndBytesConfig

# Configure 4-bit quantization
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

In [ ]:
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

# model_name = "meta-llama/Llama-3.1-8B"
model_name = "meta-llama/Llama-3.1-8B-Instruct"
# model_name = "meta-llama/Llama-2-7b"
# model_name = "google/gemma-7b-it"
# model_name = "google/gemma-3-12b-it"

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    quantization_config=quantization_config,
    device_map="auto",
)

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

In [ ]:
from langchain.llms import HuggingFacePipeline
from transformers import pipeline

# Set up LLM pipeline
query_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    temperature=0.7,
    top_p=0.95,
    max_new_tokens=200,
    return_full_text=False,
    eos_token_id=tokenizer.eos_token_id,
    do_sample=True,
)

llm = HuggingFacePipeline(
    pipeline=query_pipeline,
)

Device set to use cuda:0


# Test model

In [ ]:
# import time
# time_start = time.time()
# query = "What is the prize"
# response = llm(query)
# time_end = time.time()
# total_time = f"{round(time_end - time_start, 3)} sec."
# full_response =  f"Question: {query}\nAnswer: {response}\nTotal time: {total_time}"
# print(full_response)

In [ ]:
import time


def test_model(tokenizer, query_pipeline, query):
    time_start = time.time()
    sequences = query_pipeline(
        query,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=200,
    )
    time_end = time.time()
    total_time = f"{round(time_end - time_start, 3)} sec."

    question = sequences[0]["generated_text"][: len(query)]
    answer = sequences[0]["generated_text"][len(query) :]

    return f"Question: {question}\nAnswer: {answer}\nTotal time: {total_time}"


# --- Example ---
response = test_model(tokenizer, query_pipeline, "Please explain what is EU AI Act.")
print(response)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Both `max_new_tokens` (=300) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Question:  I am a non EU resident and would
Answer:  like to know if this EU AI Act is relevant to me.
The EU AI Act is a proposal for a regulation that aims to regulate the development, deployment, and use of artificial intelligence (AI) in the European Union. The proposal is part of the EU's efforts to ensure the safe and transparent use of AI, and to promote the competitiveness of the EU's digital economy.
The EU AI Act proposes to introduce a framework that includes:
1. **Risk-based approach**: The regulation would require developers and deployers of AI systems to conduct risk assessments and implement mitigation measures to ensure the safe and transparent use of AI.
2. **Key concepts**: The regulation would define key concepts, such as "high-risk AI systems," which would include AI systems that pose significant risks to the health, safety, or fundamental rights of individuals.
3. **Authorizations and notifications**: The regulation would require developers and deployers of high-ri

# RAG

In [ ]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate


def chatbot():
    # Define the prompt template for RetrievalQA
    prompt_template = (
        "You are a smartly assistant.\n\n"
        "Context information is below.\n"
        "---------------------\n"
        "{context}\n"
        "---------------------\n"
        "Based on the context above, give a short and concise answer to the question.\n"
        "Do not make up information. End the answer with 'Thanks for asking.'\n"
        "Question: {question}\n"
    )

    qa_prompt = PromptTemplate.from_template(prompt_template)
    print(qa_prompt)

    # Create QA chain with custom prompt template
    try:
        qa_chain = RetrievalQA.from_chain_type(
            llm=llm,
            chain_type="stuff",
            retriever=retriever,
            chain_type_kwargs={"prompt": qa_prompt},
        )
        return qa_chain
    except Exception as e:
        print(f"Error in LLM setup or query: {str(e)}")


qa_chain = chatbot()

input_variables=['context', 'question'] input_types={} partial_variables={} template="You are a smartly assistant.\n\nContext information is below.\n---------------------\n{context}\n---------------------\nBased on the context above, give a short and concise answer to the question.\nDo not make up information. End the answer with 'Thanks for asking.'\nQuestion: {question}\n"


In [ ]:
import time


def test_rag(qa, query):
    time_start = time.time()
    response = qa.run(query)
    final_answer = (
        response.lower().split("thanks for asking")[0].strip() + " thanks for asking."
    )
    time_end = time.time()
    total_time = f"{round(time_end - time_start, 3)} sec."

    print(f"{final_answer}\nTotal time: {total_time}")

In [ ]:
query = "What is the prize for the competition?"
test_rag(qa_chain, query)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


answer: the prize is $500,000, thanks for asking.
Total time: 15.934 sec.


In [ ]:
query = "Get the sentence contain the word 'Tesla' and it's timestamp."
test_rag(qa_chain, query)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


answer:
the sentence contain the word 'tesla' is:  plus these two teslas for free.
the timestamp is: [00:16:31 - 00:16:33] thanks for asking.
Total time: 16.489 sec.


In [ ]:
query = "Explain why the word 'Tesla' that is mentioned in the competition?"
test_rag(qa_chain, query)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


answer:
the word 'tesla' is mentioned in the competition because suzy is literally talking about it for nine days straight. thanks for asking.
Total time: 3.539 sec.


In [ ]:
query = "Who are the participants"
test_rag(qa_chain, query)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


answer: bailey and susie, thanks for asking.
Total time: 15.849 sec.


In [ ]:
query = "Who is the master of ceremonies of this competition?"
test_rag(qa_chain, query)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


answer: the master of ceremonies is the person who is speaking to the contestants, introducing the challenges and twists, and explaining the rules. thanks for asking.
Total time: 3.617 sec.


In [ ]:
query = "What is the host of the competition name?"
test_rag(qa_chain, query)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


answer: the name of the host is not mentioned in the provided context. thanks for asking.
Total time: 2.764 sec.


In [ ]:
query = "Did Bailey and Susie win in the end of this show?"
test_rag(qa_chain, query)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


answer: the show did not provide information about the end result of the experiment. thanks for asking.
Total time: 15.862 sec.


In [ ]:
query = "Who are 'Carl and Chris'?"
test_rag(qa_chain, query)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


answer: youtube hosts. thanks for asking.
Total time: 15.882 sec.


In [ ]:
query = "Who won at the end of the competition?"
test_rag(qa_chain, query)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


answer: there is no clear information in the provided context about the winner of the competition. thanks for asking.
Total time: 15.933 sec.
